In [1]:
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
from ipywidgets import Output
import pandas as pd
import numpy as np
import json
import os
import re
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
PROJECT_PATH = '../../Milestone3/'
#PROJECT_PATH = "/Users/tristanmartin/Desktop/UdeM_PhD/Cours/A2023/IFT6758/Project/Milestone_1/"

In [3]:
def load_data(year, season, game_ID):
    
    data_path = PROJECT_PATH+f"data/IFT6758_Data/{year}/{season}/"
    files = sorted(glob.glob(data_path+'*.json'), key=lambda x: int(re.search(r'(\d+).json', x).group(1)))
    _, filename = os.path.split(files[game_ID-1])

    with open(os.path.join(data_path, filename), 'r') as file:
        game_data = json.load(file)
    
    gameTime = game_data['gameData']['datetime']['dateTime']
    homeTeam = game_data['gameData']['teams']['home']['name']
    awayTeam = game_data['gameData']['teams']['away']['name']
    liveData = game_data['liveData']['linescore']['periods']
    num_periods = len(liveData)
    home_SoG = np.sum([liveData[i]['home']['shotsOnGoal'] for i in range(num_periods)])
    home_goals = np.sum([liveData[i]['home']['goals'] for i in range(num_periods)])
    away_SoG = np.sum([liveData[i]['away']['shotsOnGoal'] for i in range(num_periods)])
    away_goals = np.sum([liveData[i]['away']['goals'] for i in range(num_periods)])
    
    d = {'Home': [homeTeam, home_goals, home_SoG], 'Away': [awayTeam, away_goals, away_SoG]}
    df = pd.DataFrame(data=d, index = ['Teams:', 'Goals', 'SOG'])
    
    print('Game time: '+ gameTime)
    print('Game ID: '+ str(game_ID))
    print(df)
    
    return game_data

# Initializing widgets
year_slider = widgets.IntSlider(
                value=2022, 
                min=2016, 
                max=2020, 
                step=1, 
                description='Year:')

season_dropdown = widgets.Dropdown(
                options=['regular', 'playoffs'], 
                value='regular', 
                description='Season:')

game_slider = widgets.IntSlider(
                value=1,
                min=1, 
                max=1, 
                step=1, 
                description='Game ID:')

load_button = widgets.Button(description="Load Game")

play_slider = widgets.IntSlider(
                value=1,
                min=1, 
                max=1, 
                step=1, 
                description='Play ID:')

def year_slider_updated(IntSlideValue):
    year = IntSlideValue.new
    if hasattr(season_dropdown, 'value'):
        season = season_dropdown.value
        data_path = PROJECT_PATH+f"data/IFT6758_Data/{year}/{season}/"
        num_games = len(os.listdir(data_path))
        game_slider.max = num_games
        game_slider.value = 1

def season_dropdown_updated(IntSlideValue):
    if hasattr(year_slider, 'value'):
        season = IntSlideValue.new
        year = year_slider.value
        data_path = PROJECT_PATH+f"data/IFT6758_Data/{year}/{season}/"
        num_games = len(os.listdir(data_path))
        game_slider.max = num_games
        game_slider.value = 1

def game_slider_updated(IntSlideValue):
    game_number = IntSlideValue.new
    year = year_slider.value
    season = season_dropdown.value
    data_path = PROJECT_PATH+f"data/IFT6758_Data/{year}/{season}/"

def load_button_click(IntSlideValue):
    year = year_slider.value
    season = season_dropdown.value
    game_id = game_slider.value
    global data
    data = load_data(year, season, game_id)
    num_plays = len(data['liveData']['plays']['allPlays'])
    play_slider.max = num_plays
    play_slider.value = 1

output_widget = Output()

def play_slider_change(IntSlideValue):
    play_id = IntSlideValue.new
    
    with output_widget:
        output_widget.clear_output(wait=True) 
        
        play = data['liveData']['plays']['allPlays'][play_id - 1]
        play_coordinates = play['coordinates']
        play_description = play['result']['description']
        period = play['about']['period']
        periodTime = play['about']['periodTime']
        
        homeTeam_triCode = data['gameData']['teams']['home']['triCode']
        awayTeam_triCode = data['gameData']['teams']['away']['triCode']
        
        img = mpimg.imread(PROJECT_PATH+"visualizations/nhl_rink.png")
        
        plt.figure(figsize=(10, 8))
        plt.imshow(img, extent = [-100, 100, -42.5, 42.5])
        if len(play_coordinates) > 0:
            plt.scatter(play_coordinates['x'], play_coordinates['y'], s = 30)
        plt.axis()
        plt.title(play_description+f', {periodTime} P-{period}', fontsize = 16)
        plt.xlabel('feet', fontsize = 13)
        plt.ylabel('feet', fontsize = 13)
        plt.show()
        
        print(play)
    
year_slider.observe(year_slider_updated, names='value')
game_slider.observe(game_slider_updated, names='value')
season_dropdown.observe(season_dropdown_updated, names='value')
load_button.on_click(load_button_click)
play_slider.observe(play_slider_change, names='value')

widget_container = widgets.VBox([year_slider, season_dropdown, game_slider, load_button, play_slider])
display(widgets.VBox([widget_container, output_widget]))


Game time: 2021-01-13T22:30:00Z
Game ID: 1
                       Home                 Away
Teams:  Philadelphia Flyers  Pittsburgh Penguins
Goals                     6                    3
SOG                      27                   34


Game time: 2021-05-18T02:00:00Z
Game ID: 24
                      Home             Away
Teams:  Colorado Avalanche  St. Louis Blues
Goals                    4                1
SOG                     50               23


Game time: 2021-04-20T22:30:00Z
Game ID: 483
                  Home           Away
Teams:  Buffalo Sabres  Boston Bruins
Goals                0              2
SOG                 32             41
